<a href="https://colab.research.google.com/github/Namtk214/AIO-Homework/blob/main/Softmax_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit Card Fraud Detection

In [16]:
!gdown 1NwikiUU_Yzom-l4Kwy24Cn-bG46gMmG4

Downloading...
From (original): https://drive.google.com/uc?id=1NwikiUU_Yzom-l4Kwy24Cn-bG46gMmG4
From (redirected): https://drive.google.com/uc?id=1NwikiUU_Yzom-l4Kwy24Cn-bG46gMmG4&confirm=t&uuid=cf2ac902-8e33-4be1-8f23-fc27bc3e3b23
To: /content/creditcard.csv
100% 151M/151M [00:00<00:00, 210MB/s]


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [18]:
dataset_path = '/content/creditcard.csv'
df = pd.read_csv(dataset_path)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [19]:
dataset_arr = df.to_numpy()
X, y = dataset_arr[:, :-1].astype(np.float64), dataset_arr[:, -1].astype(np.int8)

In [20]:
intercept = np.ones((X.shape[0], 1))
#X = np.hstack((intercept, X))
X_b = np.concatenate((intercept, X), axis=1)

In [21]:
n_classes = np.unique(y, axis=0).shape[0]
n_samples = y.shape[0]
y_encoded = np.array([np.zeros(n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1

In [22]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(X_b, y_encoded, test_size=val_size, random_state=random_state, shuffle=is_shuffle)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle)

In [23]:
normalizer = StandardScaler()
X_train[:, 1:] = normalizer.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalizer.transform(X_val[:, 1:])
X_test[:, 1:] = normalizer.transform(X_test[:, 1:])

In [24]:
def softmax(z):
  exp_z = np.exp(z)
  return exp_z / exp_z.sum(axis=1)[:, None]

In [25]:
def predict(X, theta):
  z = np.dot(X, theta)
  return softmax(z)

In [26]:
def compute_loss(y_hat, y):
  n = y.size
  return (-1 / n) * np.sum(y * np.log(y_hat))

In [27]:
def compute_gradient(X, y, y_hat):
  n = y.size
  return (1 / n) * np.dot(X.T, (y_hat - y))

In [28]:
def update_theta(theta, gradient, learning_rate):
  return theta - learning_rate * gradient

In [29]:
def compute_accuracy(y_hat, y):
  y_hat = predict(X, theta)
  acc = (np.argmax(y_hat, axis=1) == np.argmax(y, axis=1)).mean()
  return acc


In [30]:
lr = 0.01
epochs = 30
batch_size = 1024
n_features = X_train.shape[1]

np.random.seed(42)
theta = np.random.uniform(n_features, n_classes)

In [31]:
# train_accs = []
# train_losses = []
# val_accs = []
# val_losses = []

# for epoch in range(epochs):
#     train_batch_losses = []
#     train_batch_accs = []
#     val_batch_losses = []
#     val_batch_accs = []

#     for i in range(0, X_train.shape[0], batch_size):
#         X_i = X_train[i:i+batch_size]
#         y_i = y_train[i:i+batch_size]

#         # Forward pass
#         y_hat = predict(X_i, theta)
#         train_loss = compute_loss(y_hat, y_i)

#         # Backward pass
#         gradient = compute_gradient(X_i, y_i, y_hat)
#         theta = update_theta(theta, gradient, lr)

#         train_batch_losses.append(train_loss)
#         train_acc = compute_accuracy(X_train, y_train, theta)
#         train_batch_accs.append(train_acc)

#     # Validation pass
#     y_val_hat = predict(X_val, theta)
#     val_loss = compute_loss(y_val_hat, y_val)
#     val_batch_losses.append(val_loss)

#     val_acc = compute_accuracy(X_val, y_val, theta)
#     val_batch_accs.append(val_acc)

#     # Compute average losses and accuracies for the epoch
#     train_batch_loss = sum(train_batch_losses) / len(train_batch_losses)
#     val_batch_loss = sum(val_batch_losses) / len(val_batch_losses)
#     train_batch_acc = sum(train_batch_accs) / len(train_batch_accs)
#     val_batch_acc = sum(val_batch_accs) / len(val_batch_accs)

#     # Append epoch results to lists
#     train_losses.append(train_batch_loss)
#     val_losses.append(val_batch_loss)
#     train_accs.append(train_batch_acc)
#     val_accs.append(val_batch_acc)

#     print(f'\nEPOCH {epoch + 1}:\tTraining loss: {train_batch_loss:.3f}\tValidation loss: {val_batch_loss:.3f}')


In [32]:
X_i = X_train[0:1]
y_i = y_train[0:1]
y_hat = predict(X_i, theta)
loss = compute_loss(y_hat.T, y_i)
y_hat.shape
#y_i.shape
#gradient = compute_gradient(X_i, y_i, y_hat)
#theta = update_theta(theta, gradient, lr)


(1, 31)

In [33]:
n_classes = np.unique(y, axis=0).shape[0]
n_samples = y.shape[0]
y_encoded = np.array([np.zeros(n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1

In [34]:
y_hat = np.array((0.4, 0.15, 0.05, 0.4))
y = np.array((1, 0, 0, 0))
n_classes = np.unique(y, axis=0).shape[0]
n_samples = y.shape[0]
y_encoded = np.array([np.zeros(n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1
loss = compute_loss(y_hat, y)
loss

0.22907268296853875

In [35]:
#softmax(np.array([-1, -2, 3, 2]))

sentiment analysis using pytorch

In [36]:
!gdown 1GR3IwbvKNuiVXN5E5eMGyEQtT4pP7kCt

Downloading...
From: https://drive.google.com/uc?id=1GR3IwbvKNuiVXN5E5eMGyEQtT4pP7kCt
To: /content/Twitter_Data.csv
100% 20.9M/20.9M [00:00<00:00, 101MB/s]


In [37]:
import pandas as pd
import numpy as np
import re
import nltk

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
nltk . download ('stopwords')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


dataset_path = '/content/Twitter_Data.csv'
df = pd.read_csv(dataset_path)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [38]:
df = df.dropna()


In [39]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def text_normalize(text):
    # Lowercasing
    text = text.lower()

    # Retweet old acronym "RT" removal
    text = re.sub(r'^rt\s+', '', text)

    # Hyperlinks removal
    text = re.sub(r'https?://\S+', '', text)

    # Punctuation removal
    text = re.sub(r'[^\w\s]', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)

    # Stemming
    stemmer = SnowballStemmer('english')
    words = text.split()
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)

    return text


In [40]:
df['clean_text'] = df['clean_text'].apply(lambda x: text_normalize(str(x)) if pd.notna(x) else '')


In [41]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['clean_text']).toarray()
y = df['category']

In [42]:
intercept = np.ones((X.shape[0], 1))

X_b = np.concatenate((intercept, X), axis=1)

In [43]:
n_classes = df['category'].nunique()
n_samples = df['category'].size

y = df['category'].to_numpy() + 1
y = y.astype(np.uint8)
y_encoded = np.array([np.zeros(n_classes) for _ in range(n_samples)])
y_encoded[np.arange(n_samples), y] = 1

In [44]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y_encoded, dtype=torch.float32)

In [45]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=random_state, shuffle=is_shuffle)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle)

In [46]:
X_train.shape[0]

114078

In [47]:
class SoftmaxRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SoftmaxRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=True)

    def forward(self, x):
        return self.linear(x)

In [48]:
lr = 0.1
epochs = 500
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

model = SoftmaxRegression(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)




In [ ]:
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_hat = model(X_train)
    train_loss = criterion(y_hat, y_train)
    train_losses.append(train_loss.item())
    train_acc = compute_accuracy(y_train, y_hat)
    train_accs.append(train_acc)
    train_loss.backward()
    optimizer.step()
    model.eval()
    with torch.no_grad():
        y_val_hat = model(X_val)
        val_loss = criterion(y_val_hat, y_val)
        val_loss.append(val_loss.item())
        val_acc = accuracy_score(y_val, y_val_hat)
        val_accs.append(val_acc)
    print(f'\nEPOCH {epoch + 1}:\tTraining loss: {train_loss:.3f}\tValidation loss: {val_loss:.3f}')